In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Fetch the base URL
base_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
# Make a GET request to the base URL
data = requests.get(base_url)
time.sleep(5)

In [5]:
# Parse the HTML content
soup = BeautifulSoup(data.text)
# Find the standings table
standings = soup.select('table.stats_table')[0]
# Extract links for each team
links = standings.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
# Construct URLs for each team's page
teams = [f"https://fbref.com{l}" for l in links]

In [7]:
# Make a GET request to the first team's page
data = requests.get(teams[0])
time.sleep(5)

In [8]:
# Read the table containing matches and fixtures
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [9]:
# Parse the HTML content again
soup = BeautifulSoup(data.text)
# Extract links for shooting statistics
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
# Make a GET request to the shooting statistics page
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
# Read the shooting statistics table
shooting_table = pd.read_html(data.text, match="Shooting")[0]

In [12]:
shooting_table.head()

For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2022-07-30  17:00  Community Shield  FA Community Shield  Sat   
1          2022-08-07  16:30    Premier League          Matchweek 1  Sun   
2          2022-08-13  15:00    Premier League          Matchweek 2  Sat   
3          2022-08-21  16:30    Premier League          Matchweek 3  Sun   
4          2022-08-27  15:00    Premier League          Matchweek 4  Sat   

                                         ... Standard               Expected  \
     Venue Result GF GA        Opponent  ...     Dist   FK PK PKatt       xG   
0  Neutral      L  1  3       Liverpool  ...      NaN  NaN  0     0      NaN   
1     Away      W  2  0        West Ham  ...     18.7  1.0  1     1      2.2   
2     Home      W  4  0     Bournemouth  ...     17.5  0.0  0     0      1.7   
3     Away      D  3  3   Newcastle Utd  ...     16.2  1.0  0     0      2.1   
4     Home      W  4  2  Crystal Palace  ...     14.1  0.0  0     0      2.2   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  1.4    0.11 -0.2    -0.4        Match Report  
2  1.7    0.09  1.3     1.3        Match Report  
3  2.1    0.10  0.9     0.9        Match Report  
4  2.2    0.13  1.8     1.8        Match Report  

[5 rows x 26 columns]

In [13]:
# Modify the column names for shooting_table
shooting_table.columns = shooting_table.columns.droplevel()

In [14]:
# Merge matches and shooting_table on the "Date" column
team_table = matches.merge(shooting_table[['Date','Sh','SoT','Dist','FK','PK','PKatt']], on="Date")

In [15]:
team_table.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-07-30,17:00,Community Shield,FA Community Shield,Sat,Neutral,L,1.0,3.0,Liverpool,...,4-3-3,Craig Pawson,Match Report,NaN,14,8,NaN,NaN,0,0
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,4-3-3,Michael Oliver,Match Report,NaN,13,1,18.7,1.0,1,1
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,4-2-3-1,David Coote,Match Report,NaN,19,7,17.5,0.0,0,0
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,4-3-3,Jarred Gillett,Match Report,NaN,21,10,16.2,1.0,0,0
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,4-2-3-1,Darren England,Match Report,NaN,18,5,14.1,0.0,0,0


In [16]:
# Fetch years in reverse order
years = list(range(2022, 2017, -1))

# Store all matches data
all_matches = []

In [18]:
# Iterate over years and teams
for year in years:
    # Make a GET request to the base URL
    data = requests.get(base_url)
    
    # Parse the HTML content
    soup = BeautifulSoup(data.text)
    
    # Find the standings table
    standings = soup.select('table.stats_table')[0]

    # Extract links for each team
    links = [l.get("href") for l in standings.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    
    # Construct URLs for each team's page
    teams = [f"https://fbref.com{l}" for l in links]
    
    # Extract previous season URL
    prev_season = soup.select("a.prev")[0].get("href")
    
    # Update the base URL with the previous season URL
    base_url = f"https://fbref.com{prev_season}"
    
    # Iterate over teams
    for team_url in teams:
        # Extract team name from the URL
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        # Make a GET request to the team's page
        data = requests.get(team_url)
        
        # Read the table containing matches and fixtures
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        
        # Parse the HTML content again
        soup = BeautifulSoup(data.text)
        
        # Extract links for shooting statistics
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        
        # Make a GET request to the shooting statistics page
        data = requests.get(f"https://fbref.com{links[0]}")
        
        # Read the shooting statistics table
        shooting_table = pd.read_html(data.text, match="Shooting")[0]
        
        # Modify the column names for shooting_table
        shooting_table.columns = shooting_table.columns.droplevel()
        
        try:
            # Merge matches and shooting_table on the "Date" column
            team_table = matches.merge(shooting_table[['Date','Sh','SoT','Dist','FK','PK','PKatt']], on="Date")
        except ValueError:
            continue
        
        # Filter team_table for Premier League matches only
        team_table = team_table[team_table["Comp"] == "Premier League"]
        
        # Add "Season" and "Team" columns
        team_table["Season"] = year
        team_table["Team"] = team_name
        
        # Append team_table to all_matches list
        all_matches.append(team_table)
        
        # Sleep for 5 seconds before making the next request
        time.sleep(5)

In [20]:
# Concatenate all matches data into a single dataframe
fullmatch_df = pd.concat(all_matches)


In [21]:
# Lowercase the column names
fullmatch_df.columns = [c.lower() for c in fullmatch_df.columns]

In [22]:
fullmatch_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1.0,1.0,1.0,2022,Manchester City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0.0,0.0,0.0,2022,Manchester City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,1.0,0.0,0.0,2022,Manchester City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0.0,0.0,0.0,2022,Manchester City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0.0,0.0,0.0,2022,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0,4,Tottenham,...,Match Report,NaN,7.0,1.0,18.9,1.0,0.0,0.0,2018,Huddersfield Town
36,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1,2,Watford,...,Match Report,NaN,13.0,3.0,18.1,1.0,0.0,0.0,2018,Huddersfield Town
37,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0,5,Liverpool,...,Match Report,NaN,5.0,1.0,21.6,0.0,0.0,0.0,2018,Huddersfield Town
38,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1,1,Manchester Utd,...,Match Report,NaN,7.0,3.0,19.0,1.0,0.0,0.0,2018,Huddersfield Town


In [23]:
# Save the dataframe to a CSV file named "matches4.csv"
fullmatch_df.to_csv("matches4.csv")